# SPIDS Automatic Report Generation Tutorial

Learn how to generate comprehensive HTML and PDF reports for publications and presentations.

This tutorial covers:
1. Generating single-experiment reports
2. Multi-experiment comparison reports
3. Customizing report templates
4. Export formats (HTML, PDF)
5. Publication-quality figures

**Estimated time**: 25-30 minutes

**Requirements**: 
- Jinja2 (included in dependencies)
- WeasyPrint (optional, for PDF generation)
- At least one completed experiment in `runs/` directory

## 1. Setup and Imports

In [ ]:
# Standard imports
import sys
from pathlib import Path


sys.path.insert(0, "../..")

# SPIDS imports
from prism.reporting.generator import ReportGenerator


# Check available experiments
runs_dir = Path("../../runs")
experiments = list(runs_dir.glob("*/checkpoint.pt"))

print(f"✓ Found {len(experiments)} experiments in runs/")
if len(experiments) > 0:
    print("\nAvailable experiments:")
    for exp in experiments[:5]:
        print(f"  - {exp.parent.name}")
    if len(experiments) > 5:
        print(f"  ... and {len(experiments) - 5} more")
else:
    print("\n⚠️  No experiments found. Run a training first:")
    print("     cd ../.. && uv run python main.py --obj_name europa --n_samples 64 --max_epochs 10")

# Check WeasyPrint availability
try:
    import weasyprint  # noqa: F401

    pdf_available = True
    print("\n✓ WeasyPrint installed - PDF generation available")
except ImportError:
    pdf_available = False
    print("\n⚠️  WeasyPrint not installed - HTML only")
    print("   Install with: uv add weasyprint")

## 2. Report Features Overview

SPIDS reports include:

### Executive Summary
- Final metrics (loss, SSIM, PSNR)
- Training duration and convergence
- Key configuration highlights

### Configuration Details
- Complete parameter table
- Object parameters (wavelength, distance, size)
- Telescope parameters (samples, SNR, pattern)
- Training parameters (epochs, learning rate, optimizer)

### Training History
- Training curves (loss, SSIM, PSNR vs epoch)
- Convergence analysis
- Learning rate schedule
- Memory usage statistics

### Results
- Ground truth vs reconstruction comparison
- Difference maps with error visualization
- K-space coverage heatmaps
- Sampling pattern visualization

### Appendix (Optional)
- Full configuration YAML
- Sample positions table
- Detailed metric history
- Hardware information

## 3. Initialize Report Generator

In [ ]:
# Initialize generator
template_dir = Path("../../spids/reporting/templates")
generator = ReportGenerator(template_dir=template_dir)

print("✓ Report generator initialized")
print(f"  Template directory: {template_dir}")

# Create output directory for this tutorial
output_dir = Path("./tutorial_reports")
output_dir.mkdir(exist_ok=True)
print(f"  Output directory: {output_dir.absolute()}")

## 4. Generate Single-Experiment Report

Let's generate a report for a single experiment:

In [ ]:
if len(experiments) > 0:
    # Select first available experiment
    experiment_path = experiments[0].parent

    print(f"Generating report for: {experiment_path.name}\n")

    # Generate HTML report
    html_output = output_dir / f"{experiment_path.name}_report.html"

    generator.generate_html(experiment_paths=[experiment_path], output_path=html_output)

    print(f"✓ HTML report generated: {html_output}")
    print(f"  File size: {html_output.stat().st_size / 1024:.1f} KB")
    print(f"\n  Open in browser: file://{html_output.absolute()}")
else:
    print("No experiments available. Run a training first.")

### View the Report

Click the file path above to open the report in your browser.

The report includes:
- **Header**: Experiment name, date, SPIDS version
- **Executive Summary**: Key metrics table
- **Configuration**: Complete parameter grid
- **Training History**: Interactive charts (if Plotly data available)
- **Results**: Visualizations with 300 DPI quality

## 5. Generate PDF Report

PDF reports are ideal for publications and presentations:

In [ ]:
if len(experiments) > 0 and pdf_available:
    experiment_path = experiments[0].parent

    print(f"Generating PDF report for: {experiment_path.name}\n")

    # Generate PDF report
    pdf_output = output_dir / f"{experiment_path.name}_report.pdf"

    generator.generate_pdf(experiment_paths=[experiment_path], output_path=pdf_output)

    print(f"✓ PDF report generated: {pdf_output}")
    print(f"  File size: {pdf_output.stat().st_size / 1024:.1f} KB")
    print("  Pages: ~5-8 (depending on content)")
elif not pdf_available:
    print("⚠️  WeasyPrint not installed. Install with:")
    print("   cd ../.. && uv add weasyprint")
else:
    print("No experiments available.")

### PDF Report Features:
- **High DPI**: 300 DPI figures for publication quality
- **Print optimization**: Proper page breaks and margins
- **Professional typography**: Clean fonts and spacing
- **File size**: Typically 2-5 MB depending on content

### Use Cases:
- Journal paper submissions
- Conference presentations
- Progress reports
- Archival documentation

## 6. Multi-Experiment Comparison Report

Compare multiple experiments in a single report:

In [ ]:
if len(experiments) >= 2:
    # Select first 3 experiments (or fewer if not available)
    selected_exps = [exp.parent for exp in experiments[:3]]

    print(f"Generating comparison report for {len(selected_exps)} experiments:\n")
    for exp in selected_exps:
        print(f"  - {exp.name}")
    print()

    # Generate comparison HTML report
    comparison_output = output_dir / "comparison_report.html"

    generator.generate_html(experiment_paths=selected_exps, output_path=comparison_output)

    print(f"✓ Comparison report generated: {comparison_output}")
    print(f"  File size: {comparison_output.stat().st_size / 1024:.1f} KB")
    print(f"\n  Open in browser: file://{comparison_output.absolute()}")
elif len(experiments) == 1:
    print("Only 1 experiment available. Need 2+ for comparison.")
else:
    print("No experiments available.")

### Comparison Report Sections:

1. **Summary Table**: All experiments side-by-side
2. **Metric Comparison**: Bar charts and rankings
3. **Training Curves**: Overlaid plots
4. **Configuration Diff**: Parameter differences highlighted
5. **Reconstruction Gallery**: All results displayed

### Analysis Questions:
- Which configuration achieved best metrics?
- How do training times compare?
- What parameters differ?
- Which sampling pattern works best?
- Is there a quality vs speed tradeoff?

## 7. Custom Report Templates

You can create custom Jinja2 templates for branded reports:

In [ ]:
# Create a simple custom template
custom_template = """
<!DOCTYPE html>
<html>
<head>
    <title>{{ experiments[0].name }} - Custom Report</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 900px;
            margin: 0 auto;
            padding: 20px;
            background-color: #f0f0f0;
        }
        .header {
            background-color: #2c3e50;
            color: white;
            padding: 20px;
            border-radius: 8px;
            margin-bottom: 20px;
        }
        .metric {
            background-color: white;
            padding: 15px;
            margin: 10px 0;
            border-radius: 5px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }
        .metric-value {
            font-size: 2em;
            font-weight: bold;
            color: #3498db;
        }
    </style>
</head>
<body>
    <div class="header">
        <h1>SPIDS Custom Report</h1>
        <p>Experiment: {{ experiments[0].name }}</p>
        <p>Generated: {{ timestamp }}</p>
    </div>

    {% for exp in experiments %}
    <div class="metric">
        <h2>{{ exp.name }}</h2>
        <p>Final Loss: <span class="metric-value">{{ "%.4f"|format(exp.metrics.final_loss) }}</span></p>
        <p>SSIM: <span class="metric-value">{{ "%.4f"|format(exp.metrics.ssim) }}</span></p>
        <p>PSNR: <span class="metric-value">{{ "%.2f"|format(exp.metrics.psnr) }} dB</span></p>
    </div>
    {% endfor %}
</body>
</html>
"""

# Save custom template
custom_template_path = output_dir / "custom_template.html"
custom_template_path.write_text(custom_template)

print(f"✓ Custom template created: {custom_template_path}")
print("\nTo use custom template:")
print("  spids report runs/experiment --template custom_template.html")

### Available Template Variables:

```python
experiments = [{
    'name': str,
    'path': Path,
    'metrics': {
        'final_loss': float,
        'ssim': float,
        'psnr': float,
        'training_time': float,
        'epochs': int
    },
    'config': dict,
    'training_history': {
        'epoch': list,
        'loss': list,
        'ssim': list,
        'psnr': list
    }
}]

timestamp: str  # ISO format
summary: dict  # Statistics across experiments
figures: dict  # Base64-encoded images
include_appendix: bool
```

## 8. Command-Line Report Generation

For production use, the CLI is recommended:

### Single Experiment
```bash
# Generate HTML report
cd ../..
spids report runs/experiment --format html

# Generate PDF report
spids report runs/experiment --format pdf --output report.pdf

# Specify output location
spids report runs/experiment --format html --output docs/my_report.html
```

### Multi-Experiment Comparison
```bash
# Compare multiple experiments
spids report runs/exp1 runs/exp2 runs/exp3 --format pdf --output comparison.pdf

# Use wildcards
spids report runs/europa_* --format html
```

### Custom Templates
```bash
# Use custom template
spids report runs/experiment --template custom_template.html --output custom_report.html
```

## 9. Batch Report Generation

Generate reports for all experiments:

In [ ]:
if len(experiments) > 0:
    print(f"Generating reports for all {len(experiments)} experiments...\n")

    batch_output_dir = output_dir / "batch_reports"
    batch_output_dir.mkdir(exist_ok=True)

    for i, exp in enumerate(experiments[:3], 1):  # Limit to 3 for demo
        exp_path = exp.parent
        output_path = batch_output_dir / f"{exp_path.name}.html"

        try:
            generator.generate_html(experiment_paths=[exp_path], output_path=output_path)
            print(f"[{i}/{min(3, len(experiments))}] ✓ {exp_path.name}")
        except Exception as e:
            print(f"[{i}/{min(3, len(experiments))}] ✗ {exp_path.name}: {e}")

    print(f"\n✓ Batch reports saved to: {batch_output_dir}")
    print(f"  Total reports: {len(list(batch_output_dir.glob('*.html')))}")
else:
    print("No experiments available.")

## 10. Publication Workflow

### Step 1: Run Experiments
```bash
# Run systematic study
for samples in 50 100 150 200; do
    python main.py --obj_name europa --n_samples $samples \
                   --fermat --name europa_${samples}samples
done
```

### Step 2: Generate Comparison Report
```bash
# Create comparison report
spids report runs/europa_* --format pdf --output paper_figure.pdf
```

### Step 3: Extract Figures
- Open PDF in editor
- Extract individual figures
- All figures are 300 DPI (publication quality)

### Step 4: Write Paper
- Use metrics table for results section
- Include training curves as supplementary material
- Reference SPIDS in methods section

## 11. Troubleshooting

### Common Issues:

**PDF Generation Fails**
```bash
# Install WeasyPrint
cd ../..
uv add weasyprint

# On Ubuntu/Debian, may need system libraries:
sudo apt-get install python3-cffi python3-brotli libpango-1.0-0

# On macOS (requires Homebrew):
brew install pango
```

**Report Shows "No Data"**
- Ensure checkpoint.pt exists in experiment directory
- Check checkpoint contains metrics
- Verify experiment completed successfully

**Figures Not Appearing**
- Check matplotlib is installed
- Verify checkpoint contains reconstruction data
- Check console for error messages

**Large File Size**
- Reduce DPI in template (default: 300)
- Compress images before embedding
- Use PNG instead of raw arrays

## 12. Best Practices

### Report Organization:
1. **Name conventions**: Use descriptive experiment names
2. **Directory structure**: Organize reports by date/project
3. **Version control**: Track report templates in git

### Content Guidelines:
1. **Executive summary**: Keep to 1 page
2. **Figures**: Include captions and axis labels
3. **Metrics**: Show standard deviations when available
4. **Configuration**: Highlight important parameters

### Quality Assurance:
1. **Review**: Always review reports before sharing
2. **Verify metrics**: Cross-check with TensorBoard
3. **Test printing**: Ensure PDF prints correctly
4. **Accessibility**: Use colorblind-friendly palettes

## 13. Next Steps

### Explore More:
- Create custom templates for your organization
- Automate report generation in CI/CD
- Combine with dashboard for complete workflow
- Share reports with collaborators

### Related Tools:
- **Dashboard**: Real-time monitoring (Tutorial 4)
- **Compare CLI**: Terminal-based comparison
- **Inspect CLI**: Detailed checkpoint inspection
- **Animate**: Training progression videos

### Advanced Topics:
- Custom CSS styling
- JavaScript interactivity in HTML reports
- Automated email delivery
- Integration with LaTeX workflow

## Summary

You've learned:
- ✓ Generate HTML and PDF reports
- ✓ Create multi-experiment comparisons
- ✓ Customize report templates
- ✓ Use CLI for production workflows
- ✓ Best practices for publications

**Generated Reports**:
- Single experiment: `tutorial_reports/experiment_report.html`
- Comparison: `tutorial_reports/comparison_report.html`
- Batch reports: `tutorial_reports/batch_reports/`

---

**Resources**:
- [Reporting Documentation](../../docs/examples/reports/README.md)
- [Example Reports](../../docs/examples/reports/)
- [Template Reference](../../spids/reporting/templates/)
- [GitHub Issues](https://github.com/username/SPIDS/issues)